# Introduction

This file creates the (a) Boathire dataset. Each Boathire dataset contains the same process variants, but each dataset is unique, since the natural language events are created randomly.

In [1]:
import pandas as pd
# Initialise Open AI client
from openai import OpenAI
client = OpenAI()

Initialise a few variable. Events will be held in dictionaries. The event log is represented as a data frame in typical event log format.

In [ ]:
eventLog = pd.DataFrame(columns=['CaseId', 'ActivityId', 'Text', 'Activity', 'SpecificActivity'])

# Create variants 1-9 / Large Group Uncatered

## Variants 1: Large Group requests 
There are 54 traces in total to generate.
Start with Receive Enquiry / Uncatered / Large Group activity.
This could be done faster in a bulk request, but we're not doing that to avoid running into any rate limits.

### Create Receive Enquiry

In [ ]:
def createReceiveEnquiry(request):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": request
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

# testing
receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
receiveEnquiry


### Create Send Offer Event function
This function should determine automatically which boat to offer. It might get it wrong sometimes and could also offer a choice of boats. This is intentional, to add some noise to the data set.

In [ ]:
def createSendOfferEvent(receiveEnquiry):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": 
"""You are James and are working for a British boat hire company called Boathire. 
Boathire Boathire has three crewed charter boats for private events cruises: 
The Aquaholic, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. 
Each boat can be chartered with or without catering. 

Boathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfarer Cruiser. 
The Delphia 24 can carry up to 4 crew and the Wayfarer Cruiser can carry up to three crew"""
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": receiveEnquiry
            }
        ]
        }
    ],
    temperature=1,
    max_tokens=1600,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content

# test only
sendOffer = createSendOfferEvent(receiveEnquiry)
sendOffer


### Create Receive Booking function
Since it's not guaranteed that GPT only offers one boat, we need to specify what it should accept.

In [ ]:
def createReceiveBooking(re, so, boat):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Confirm the booking of the " + boat + "."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

# testing 
receiveBooking = createReceiveBooking(receiveEnquiry, sendOffer, "Pier Pressure")
receiveBooking

### Add variant 1 to event log
Create all traces for the variant

In [ ]:
def addTraceToEventLog(events, activities, specificActivities):
    """
    :param variant: the variant number of this trace
    :param events: An array containing the natural language events in order
    :param activites: An array containing the activities (classes) of the events in this trace
    :param specificActivities: Same as activities, but specifying the classes as specific as possible
    """
    if (len(events) != len(activities) != len(specificActivities)):
         raise ValueError("events, activities, and specificActivies must be of the same size")

    global eventLog
    
    caseId = len(eventLog['CaseId'].unique())

    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(len(events)),
            'Text': events,
            'Activity': activities,
            'SpecificActivity': specificActivities
        }
    )    
    eventLog = pd.concat([eventLog, df], ignore_index=True)


numberOfCases = 10
for caseId in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'Pier Pressure')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure', 
                            'Receive Booking / Pier Pressure']
    )
    
    
eventLog

## Variant 2: Reject offer
Define Create Offer Rejected function. This is not specific to any offer type.

In [ ]:
def createOfferRejected(re, so):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            #"text": "Decline/reject the offer. Make up a reason, if you want."
            "text": "Decline/reject the offer."
          }
        ]
      }
    ],
    temperature=1.3,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

createOfferRejected(receiveEnquiry, sendOffer)


### Add variant 2 to event log

In [ ]:
numberOfCases = 9
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected =  createOfferRejected(receiveEnquiry, sendOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure', 
                            'Offer Rejected']
    )
    
    
eventLog


## Variant 3
Variant 3 doesn't have an event 3 and is straight forward to add to the event log.

In [ ]:
numberOfCases = 9
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer],
        activities=['Receive Enquiry', 'Send Offer'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure']
    )
    
    
eventLog


## Variant 4
RE - SO - DR - RA - AD - Receive Booking

This variant needs new functions for
- Discount requested (event 3)
- Request Approval (event 4)
- Send revised offer with Discount (event 5)
- Receive Booking After Discount (event 6)

### Create Discout Requested
General function, independent of the enquiry type.

In [ ]:
def createDiscountRequested(re, so):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Ask for a discount."
          }
        ]
      }
    ],
    temperature=1.3,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

# testing only
test_event3 = createDiscountRequested(receiveEnquiry[0], sendOffer[0])
test_event3


### Create Request Approval function
General function independent of type (or rather, all context is read from the preceeding events).

In [ ]:
def createRequestApproval(re, so, dr):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": """You are James and are working for a British boat hire company called Boathire. 
            Boathire has three crewed charter boats for private events cruises: 
            The Pier Pressure, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. Each boat can be chartered with or without catering.
            
            Boathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfairer.
            
            When customer request discounts, this needs to be handled by your supervisor Sarah."""
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Write a quick email to your supervisor Sarah, asking her to get in contact with the customer and offer or decline the requested discount."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

# test only
test_requestApproval = createRequestApproval(sendOffer[0], receiveEnquiry[0], test_event3)
test_requestApproval


### Create Discounted Offer function
Sarah is sending revised offers, with some discount.

In [ ]:
def createDiscountedOffer(re, so, dr, ra):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": """You are Sarah and are working for a British boat hire company called Boathire. 
            You are James' supervisor.
            Boathire has three crewed charter boats for private events cruises: 
            The Pier Pressure, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. Each boat can be chartered with or without catering.
            
            Boathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfairer.
            """
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Offer a discount to the customer."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

test_event5 = createDiscountedOffer(sendOffer[0], receiveEnquiry[0], test_event3, test_requestApproval)
test_event5


### Create Receive Booking After Discount
We need to specify the boat, since multiple might be offered...

In [ ]:
def createReceiveBookingAfterDiscount(re, so, dr, ra, e5, boat):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": e5
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Confirm the booking of the " + boat + "."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

createReceiveBookingAfterDiscount(sendOffer[0], receiveEnquiry[0], test_event3, test_requestApproval, test_event5, "Pier Pressure")


### Add to event log
All functions are defined now, let's add it to the event log

In [ ]:
numberOfCases = 4
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Pier Pressure')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Uncatered / Pier Pressure / Discounted', 
                            'Receive Booking / Pier Pressure']
    )
    
    
eventLog


## Variant 5
RE - SO - DR - RA - AD - Offer Rejected

Need to create a new function to reject offer after discount

### Create Decline Offer After Discount function
General purpose.

In [ ]:
def createDeclineOfferAfterDiscount(re, so, dr, ra, e5):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": e5
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Decline/reject the offer."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

createDeclineOfferAfterDiscount(sendOffer, receiveEnquiry, discountRequested, requestApproval, sendDiscountedOffer)


### Add to event log

In [ ]:
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Uncatered / Pier Pressure / Discounted', 
                            'Offer Rejected']
    )
    
    
eventLog


## Variant 6
RE - SO - DR - RA - SD - no reply

In [ ]:
numberOfCases = 2
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Uncatered / Pier Pressure']
    )
    
eventLog


## Variant 7
RE - SO - DR - RA - DD - RO

Need a new function for Decline discount

### Create Decline Discount activity

In [ ]:
def createDeclineDiscount(re, so, dr, ra):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": """You are Sarah and are working for a British boat hire company called Boathire. 
            You are James' supervisor.
            Boathire has three crewed charter boats for private events cruises: 
            The Pier Pressure, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. Each boat can be chartered with or without catering.
            
            Boathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfairer.
            """
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Regretfully decline the discount request of the customer."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

createDeclineDiscount(sendOffer, receiveEnquiry, discountRequested, requestApproval)


### Add to event log

In [ ]:
numberOfCases = 2
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Pier Pressure')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Pier Pressure']
    )
    
eventLog


## Variant 8
RE - SO - DR - RA - DD - OR

In [ ]:
numberOfCases = 2
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )
    
    
eventLog


## Variant 9
RE - SO - DR - RA - DD - no reply

In [ ]:
numberOfCases = 2
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Uncatered / Large Group', 
                            'Send Offer / Uncatered / Pier Pressure',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount']
    )
    
    
eventLog


# Variants 10-18
RE / Uncatered / Medium Group

## Variant 10
Now that we have all function, etc... set up, we're going to do things slightly different are are creating and adding one variant in each code block.

In [ ]:
numberOfCases = 5
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking = createReceiveBooking(receiveEnquiry, sendOffer, 'Aquaholic')
    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic', 
                            'Receive Booking / Uncatered / Aquaholic']
    )
    


## Variant 11

In [ ]:
numberOfCases = 4
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected = createOfferRejected(receiveEnquiry, sendOffer)
    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic', 
                            'Offer Rejected']
    )
    
   
eventLog


## Variant 12

In [ ]:
numberOfCases = 4
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
        
    addTraceToEventLog(
        events = [receiveEnquiry, sendOffer],
        activities= ['Receive Enquiry', 'Send Offer'],
        specificActivities= ['Receive Enquiry / Uncatered / Medium Group', 
                             'Send Offer / Uncatered / Aquaholic']
    )
eventLog

## Variant 13

In [ ]:
numberOfCases = 2
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Aquaholic')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Crewed / Aquaholic / Discounted', 
                            'Receive Booking / Aquaholic']
    )



## Variant 14
Even though we have 0 cases, we're including this here, so we can copy it later...

In [ ]:
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Crewed / Aquaholic / Discounted', 
                            'Offer Rejected']
    )

## Variant 15

In [ ]:
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Crewed / Aquaholic / Discounted']
    )

## Variant 16

In [ ]:
numberOfCases = 2
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Pier Pressure')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Uncatered / Aquaholic / Undiscounted']
    )


## Variant 17

In [ ]:
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )
    


## Variant 18

In [ ]:
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Uncatered / Medium Group', 
                            'Send Offer / Uncatered / Aquaholic',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount']
    )
    


# Variants 19-27

In [ ]:
# Variant 19
numberOfCases = 7
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking = createReceiveBooking(receiveEnquiry, sendOffer, 'Harbor Harmony')
    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony', 
                            'Receive Booking / Harbor Harmony']
    )
    
# Variant 20
numberOfCases = 6
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected = createOfferRejected(receiveEnquiry, sendOffer)
    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony', 
                            'Offer Rejected']
    )
   
# Variant 21
numberOfCases = 6
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer],
        activities=['Receive Enquiry', 'Send Offer'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony']
    )
   
# Variant 22
numberOfCases = 3
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Harbor Harmony')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Uncatered / Harbor Harmony / Discounted', 
                            'Receive Booking / Harbor Harmony']
    )

# Variant 23
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Uncatered / Harbor Harmony / Discounted', 
                            'Offer Rejected']
    )

# Variant 24
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Uncatered / Harbor Harmony / Discounted']
    )

# variant 25
numberOfCases = 2
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Harbor Harmony')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Harbor Harmony']
    )

# variant 26
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )

# variant 27
numberOfCases = 1
for i in range(numberOfCases):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). No catering required - do not mention it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Uncatered / Small Group', 
                            'Send Offer / Uncatered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount']
    )


# Variants 28-36
Large Group Catered Pier Pressure

In [ ]:
createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")

In [ ]:
# Variant 28
for i in range(21):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'Pier Pressure')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Receive Booking / Pier Pressure']
    )

# Variant 29
for i in range(13):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected =  createOfferRejected(receiveEnquiry, sendOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Offer Rejected']
    )

# Variant 30
for i in range(16):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer],
        activities=['Receive Enquiry', 'Send Offer'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure']
    )

# Variant 31
for i in range(7):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Pier Pressure')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Pier Pressure / Discounted', 
                            'Receive Booking / Pier Pressure']
    )

# Variant 32
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Pier Pressure / Discounted', 
                            'Offer Rejected']
    )

# Variant 33
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Pier Pressure / Discounted']
    )

# Variant 34
for i in range(3):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Pier Pressure')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Catered / Pier Pressure']
    )

# Variant 35
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )
    
    
# Variant 36
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a large group of people (between 30 and 50, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Catered / Large Group', 
                            'Send Offer / Catered / Pier Pressure', 
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount']
    )
    
    


# Variants 37-45

In [ ]:
# Variant 37
for caseId in range(9):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'Aquaholic')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Receive Booking / Catered / Aquaholic']
    )

# Variant 38
for i in range(6):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected =  createOfferRejected(receiveEnquiry, sendOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Offer Rejected']
    )

# Variant 39
for i in range(7):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer],
        activities=['Receive Enquiry', 'Send Offer'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic']
    )

# Variant 40
for i in range(3):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Aquaholic')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Aquaholic / Discounted', 
                            'Receive Booking / Aquaholic']
    )

# Variant 41
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Aquaholic / Discounted', 
                            'Offer Rejected']
    )

# Variant 42
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Aquaholic / Discounted']
    )

# Variant 43
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Aquaholic')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Catered / Aquaholic']
    )

# Variant 44
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )
    
    
# Variant 45
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a  group of people (between 10 and 30, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Catered / Medium Group', 
                            'Send Offer / Catered / Aquaholic', 
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount']
    )
    
    


# Variants 46-55

In [ ]:
# Variant 46
for i in range(13):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'Harbor Harmony')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony', 
                            'Receive Booking / Harbor Harmony']
    )

# Variant 47
for i in range(8):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected =  createOfferRejected(receiveEnquiry, sendOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony', 
                            'Offer Rejected']
    )

# Variant 48
for i in range(10):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer],
        activities=['Receive Enquiry', 'Send Offer'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony']
    )

# Variant 49
for i in range(4):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Harbor Harmony')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Harbor Harmony / Discount', 
                            'Receive Booking / Harbor Harmony']
    )

# Variant 50
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Harbor Harmony / Discount', 
                            'Offer Rejected']
    )

# Variant 51
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Send Offer / Catered / Harbor Harmony / Discount']
    )

# Variant 52
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Harbor Harmony')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Harbor Harmony']
    )

# Variant 53
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )
    
    
# Variant 54
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
        """You want to hire a boat for a small group of people (less than 10, pick a number). Catering required - ask for it. 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequested(receiveEnquiry, sendOffer)
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Catered / Small Group', 
                            'Send Offer / Catered / Harbor Harmony',
                            'Discount Requested',
                            'Request Approval',
                            'Decline Discount']
    )
    
    


# Variants 55- 72 Sailing Boats

In [ ]:
receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
print(receiveEnquiry)

In [ ]:
sendOffer = createSendOfferEvent(receiveEnquiry)
print(sendOffer)


In [ ]:
receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'Delphia 24')
print(receiveBooking)

In [ ]:
print(createOfferRejected(receiveEnquiry, sendOffer))

In [ ]:
def createDiscountRequestedForBoat(re, so, boat):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "You would like to book the " + boat + ", but it's a bit too expensive. Ask for a discount."
          }
        ]
      }
    ],
    temperature=1.3,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

# testing only
discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Delphia 24')
print(discountRequested)



## Delphia 24 variants

In [ ]:
# Variant 55: RE - SE - RB (Delphia 24)
for caseId in range(25):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'Delphia 24')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing', 
                            'Receive Booking / Delphia 24']
    )

# Variant 56: RE - SE - RB (Wayfarer)
for caseId in range(30):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'Wayfarer Cruiser')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing', 
                            'Receive Booking / Wayfarer Cruiser']
    )


# Variant 57: RE - SE - OR
for i in range(11):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected =  createOfferRejected(receiveEnquiry, sendOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing', 
                            'Offer Rejected']
    )

# Variant 58: RE - SE 
for i in range(20):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer],
        activities=['Receive Enquiry', 'Send Offer'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing']
    )

# Variant 59: RE-SO-DR (Delphia 24)-RA-DO-RB(Delphia 24)
for i in range(4):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Delphia 24')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Delphia 24')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Delphia 24',
                            'Request Approval',
                            'Send Offer / Delphia 24 / Discounted', 
                            'Receive Booking / Delphia 24']
    )

# Variant 60: RE-SO-DR(Delphia 24)-RA-DO-OR
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Delphia 24')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Delphia 24',
                            'Request Approval',
                            'Send Offer / Delphia 24 / Discounted', 
                            'Offer Rejected']
    )

# Variant 61: RE-SO-DR(Delphia 24)-RA-DO
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Delphia 24')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Delphia 24',
                            'Request Approval',
                            'Send Offer / Delphia 24 / Discounted']
    )

# Variant 62: RE-SO-DR (Delphia 24)-RA-DD-RB(Delphia 24)
for i in range(6):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Delphia 24')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, 'Delphia 24')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Delphia 24',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Delphia 24']
    )

# Variant 63: RE-SO-DR(Delphia 24)-RA-DD-OR
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Delphia 24')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Delphia 24',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )

# Variant 64: RE-SO-DR(Delphia 24)-RA-DO
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Delphia 24')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Delphia 24',
                            'Request Approval',
                            'Decline Discount']
    )

## Wayfarer discount variants

In [ ]:
# Variant 65: RE-SO-DR (Wayfarer)-RA-DO-RB(Wayfarer)
for i in range(2):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Wayfarer Cruiser')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, 'Wayfarer Cruiser')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Wayfarer Cruiser',
                            'Request Approval',
                            'Send Offer / Wayfarer Cruiser / Discounted', 
                            'Receive Booking / Wayfarer Cruiser']
    )

# Variant 66: RE-SO-DR(Wayfarer Cruiser)-RA-DO-OR
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Wayfarer Cruiser')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Wayfarer Cruiser',
                            'Request Approval',
                            'Send Offer / Wayfarer Cruiser / Discounted', 
                            'Offer Rejected']
    )

# Variant 67: RE-SO-DR(Wayfarer Cruiser)-RA-DO
for i in range(4):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Wayfarer Cruiser')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    sendDiscountedOffer = createDiscountedOffer(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, sendDiscountedOffer],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Wayfarer Cruiser',
                            'Request Approval',
                            'Send Offer / Wayfarer Cruiser / Discounted']
    )

# Variant 68: RE-SO-DR (Wayfarer Cruiser)-RA-DD-RB(Wayfarer Cruiser)
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Wayfarer Cruiser')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    receiveBooking = createReceiveBookingAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, 'Wayfarer Cruiser')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Wayfarer Cruiser',
                            'Request Approval',
                            'Decline Discount', 
                            'Receive Booking / Wayfarer Cruiser']
    )

# Variant 69: RE-SO-DR(Wayfarer Cruiser)-RA-DD-OR
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Wayfarer Cruiser')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Wayfarer Cruiser',
                            'Request Approval',
                            'Decline Discount', 
                            'Offer Rejected']
    )

# Variant 70: RE-SO-DR(Wayfarer Cruiser)-RA-DO
for i in range(1):

    receiveEnquiry = createReceiveEnquiry(
"""You want to hire a sailing boat. Ask about suitable boats for a specific number of people (anything between 2 and 4, choose a number). 
Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    discountRequested = createDiscountRequestedForBoat(receiveEnquiry, sendOffer, 'Wayfarer Cruiser')
    requestApproval = createRequestApproval(receiveEnquiry, sendOffer, discountRequested)
    declineDiscount = createDeclineDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval)
    offerRejected = createDeclineOfferAfterDiscount(receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, discountRequested, requestApproval, declineDiscount],
        activities=['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
        specificActivities=['Receive Enquiry / Sailing', 
                            'Send Offer / Sailing',
                            'Discount Requested / Wayfarer Cruiser',
                            'Request Approval',
                            'Decline Discount']
    )

# Rowing boats

In [ ]:
receiveEnquiry = createReceiveEnquiry("""You want to hire a rowing boat. Make up the details (e.g. date, time, your name, etc.)""")
print(receiveEnquiry)


In [ ]:
sendOffer = createSendOfferEvent(receiveEnquiry)
print(sendOffer)

In [ ]:
receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'rowing boat')
print(receiveBooking)

In [ ]:
# Variant 71: RE - SE - RB
for caseId in range(13):

    receiveEnquiry = createReceiveEnquiry("""You want to hire a rowing boat. Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    receiveBooking =  createReceiveBooking(receiveEnquiry, sendOffer, 'rowing boat')

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, receiveBooking],
        activities=['Receive Enquiry', 'Send Offer', 'Receive Booking'],
        specificActivities=['Receive Enquiry / Rowing', 
                            'Send Offer / Rowing', 
                            'Receive Booking / Rowing']
    )


# Variant 72: RE - SE - OR
for i in range(5):

    receiveEnquiry = createReceiveEnquiry("""You want to hire a rowing boat. Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)
    offerRejected =  createOfferRejected(receiveEnquiry, sendOffer)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer, offerRejected],
        activities=['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
        specificActivities=['Receive Enquiry / Rowing', 
                            'Send Offer / Rowing', 
                            'Offer Rejected']
    )

# Variant 73: RE - SE 
for i in range(9):

    receiveEnquiry = createReceiveEnquiry("""You want to hire a rowing boat. Make up the details (e.g. date, time, your name, etc.)""")
    sendOffer = createSendOfferEvent(receiveEnquiry)

    addTraceToEventLog(
        events=[receiveEnquiry, sendOffer],
        activities=['Receive Enquiry', 'Send Offer'],
        specificActivities=['Receive Enquiry / Rowing', 
                            'Send Offer / Rowing']
    )


# Save

In [ ]:
eventLog.to_csv('Boathire 370.csv', index=False, encoding='utf-8-sig')

In [ ]:
print(eventLog.iloc[1].Text)

# EOF